In [1]:
import pandas as pd
import scryfall
"""
fastest probably to manually download all card file, save JSON to this folder and replace the filename below
make sure to update the file below in order to have current price and card data
price data is considered stale after 24 hours
"""

#takes a json file downloaded from scryfall
#fname = "all-cards-20210224101318.json"
fname = "default-cards-20210318210305.json"
#takes a .dek xml file exported from MTGO
#dekname = "Full_Trade_List.dek"
dekname = "Full_Trade_List-3-21.dek"
basiclands = ["Plains","Island","Swamp","Mountain","Forest"]

#todo
# 1. done (stripped done to which rows have at least either a mtgo_id or mtgo_foil_id)
# 2. done (selected which columns to keep)
# 3. done (load mtgo .dek file)
# 4. done (get prices for each item and then create a column and that dataframe with the price)
# 5. clean up and simplify the code & comment better
# 6. done (add interesting data print out in a final cell)
# 7. done(add price of tix to file)
# 8. done (fix prices for foils)
# 9. estimate prices of unlisted cards

In [3]:
#takes a while, loads in the scryfall file, drops some of the less important columns
scry_df = scryfall.read_scry(fname)
collection = scryfall.read_dek(dekname)


In [4]:
#shrinks the scryfall file to just stuff that we want to work with
simpler_cols = pd.Series(['mtgo_id','mtgo_foil_id','prices'])
small_scry = scry_df[simpler_cols]#.dropna(thresh=2,subset=["mtgo_id","mtgo_foil_id"])

In [5]:
#joining the two tables via the mtgo id value
collection = collection.set_index('mtgo_id').join(small_scry.set_index('mtgo_id'))
#when we joined the collection with the price guide, i set the index to be the mtgo ID, undoing that now
collection = collection.reset_index()

In [7]:
#some objects are missing price data in the scryfall database, here are some of them in your collection
collection[collection["prices"].isna()]

,mtgo_id,Quantity,Name,mtgo_foil_id,prices
117,13008,4,Thran Foundry,NaN,NaN
124,13476,4,Alley Grifters,NaN,NaN
513,19736,2,Viridian Longbow,NaN,NaN
756,22426,1,Rewind,NaN,NaN
826,23247,4,Shred Memory,NaN,NaN
...,...,...,...,...,...
5029,80298,1,Auspicious Starrix,NaN,NaN
5053,80418,1,Ruinous Ultimatum,NaN,NaN
5061,80490,1,Sleeper Dart,NaN,NaN
5110,81821,1,Thriving Moor,NaN,NaN


In [11]:
#for each price dict in the price column, we are extracting the 'tix' value where there is one

tix = list()
for index,row in collection.iterrows():
    if type(row['prices']) == dict:
        tix.append(float(row['prices']['tix']))
    elif row['Name'] == "Event Ticket":
        tix.append(1)
    else: #find foil price by going back and finding the row with the foil id and then getting the price from that line
        try:
            foil_price = small_scry[small_scry['mtgo_foil_id']==row['mtgo_id']]['prices'].values[0]['tix']
            tix.append(float(foil_price))
        except:
            #for now just make unlisted cards 1 cent
            tix.append(.01)
#add a new column that has the tix values for each line
collection['tix'] = pd.Series(tix)

In [13]:
# Returns total value of collection in Tix (~USD)
collection['Quantity'] = collection['Quantity'].astype('int')

print( "Value in Tix: " +str((collection['tix'] * collection['Quantity']).sum() ))

#Average Value
print("avg value:" + str(collection['tix'].mean()))
print("mode value:" + str(collection['tix'].mode()[0]))

#Most valuable cards:
print("\nTop 5 most valuable cards")
print(collection[['tix','Name']].sort_values('tix',ascending=False).head(5))

#Number of cards worth 2 cents or less (not actually the legal PD cards)
print("\nNumber of penny cards: " + str(len(collection[collection['tix']<=.02])))

#counts of unique basic lands
print("\nNumber of unique basic lands: \n" + str(collection[collection['Name'].isin(basiclands)]['Name'].value_counts()))

#most common printing
print("\nMost common printing\n" + str(collection[collection['Quantity']==collection['Quantity'].max()]['Name']))

#most common card
print("\nMost common cards\n" + str(collection.groupby('Name')['Quantity'].sum().nlargest(15)))

Value in Tix: 535.0500000000001
avg value:0.059772288691622884
mode value:0.03

Top 5 most valuable cards
        tix              Name
2918  18.45       Black Lotus
2876  15.38     Demonic Tutor
2931   8.06  Wheel of Fortune
21     5.98           Impulse
2920   5.32        Mana Vault

Number of penny cards: 1740

Number of unique basic lands: 
Forest      229
Island      222
Swamp       218
Mountain    217
Plains      217
Name: Name, dtype: int64

Most common printing
4184      Plains
4185      Island
4186       Swamp
4187    Mountain
4188      Forest
Name: Name, dtype: object

Most common cards
Name
Forest                   307
Mountain                 269
Island                   267
Plains                   263
Swamp                    260
Evolving Wilds            25
Giant Growth              19
Bloodfell Caves           17
Fog                       15
Traveler's Amulet         15
Blossoming Sands          14
Swiftwater Cliffs         14
Dismal Backwater          13
Elixir of Immo